In [38]:
import pandas as pd
import numpy as np
import re
import collections, itertools
import numpy as np
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.util import ngrams
from nltk.probability import FreqDist
from sklearn.feature_extraction.text import TfidfVectorizer
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/tatianacogne/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/tatianacogne/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/tatianacogne/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/tatianacogne/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [14]:
with open('data/text_article_4507624.txt') as f:
    text = f.read()

0. Function that transforms a list of strings into 1 big concatenated string and vice-versa

In [15]:
def list_to_text(list_input, stops = []):
    text_output = ' '.join([word for word in list_input if word not in stops]) 
    return text_output

def text_to_list(text_input):
    list_output = word_tokenize(text_input)
    return list_output

1. Ponctuation 

In [16]:
text = re.sub('[^A-Za-z0-9]+', ' ', text)

2. Lower all words

In [17]:
text = text.lower()

3. Remove stopwords

In [18]:
stop_words = set(stopwords.words('english'))
word_tokens = word_tokenize(text)
filtered_text = [w for w in word_tokens if not w.lower() in stop_words]

print('Number of words before removing the stop words',len(word_tokens))
print('Number of words after removing the stop words',len(filtered_text))

Number of words before removing the stop words 6192
Number of words after removing the stop words 3165


4. Lemmatization

In [19]:
wordnet_lemmatizer = WordNetLemmatizer()

In [20]:
# Lemmatize only the noun
x = [wordnet_lemmatizer.lemmatize(word, pos='n') for word in filtered_text]

5. N-Gram 

In [21]:
def ngrams_list(n):
    """
        Compute ngrams.
        
        Args:
            n (int): the number of words to words to assemble in the ngram.
        
        Returns :
            A list composed of the ngrams.
    """
    m = []
    nx_grams = ngrams(sequence = nltk.word_tokenize(text), n = n)
    for gram in nx_grams:
        m.append(gram)
    return m

6. Remove most frequent and least frequent words

In [22]:
def remove(a,l,h):
    """
        Remove most and least frequent words.
        Args:
            a (list) : list on which operations should be made.
            l (float): the proportion of top l% least frequent words to remove from the numer of different words.
            h (float): the proportion of top h% most frequent words to remove from the numer of different words.
    
        Returns:
            A copy of the input text without frequent and infrequent words.
    """
    f = FreqDist(a)

    df_fdist = pd.DataFrame({'Word': f.keys(), 'Number of apparitions': f.values()})
    L= l*len(df_fdist)
    L=int(L)

    H=h*len(df_fdist)
    H=int(H)
    
    df_fdesc = df_fdist.sort_values(by='Number of apparitions', ascending=False)
    df_fasc = df_fdist.sort_values(by='Number of apparitions', ascending=True)

    most_freq_words_list = list(df_fdesc['Word'][:H])
    least_freq_word_list = list(df_fasc['Word'][:L])
    stopwords = most_freq_words_list + least_freq_word_list
    textlist_wo_extremes = list_to_text(filtered_text, stopwords)
    #text_wo_extremes = ' '.join([word for word in filtered_text if word not in stopwords]) 

    return textlist_wo_extremes, df_fdist

In [23]:
result = remove(filtered_text,0.03,0.06)

In [13]:
result_1 = result

In [24]:
result_2 = result

## TFIDF
https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html

In [202]:
corpus = [result_1[0],result_2[0]]
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus)
X.shape

(2, 1818)

## Latent Semantic Analysis (LSA)
Using the TFIDF matrix, perform an SVD to keep only the k topics 

In [203]:
from sklearn.decomposition import TruncatedSVD

In [215]:
# Define the number of topics or components
num_components=50

# Create SVD object
lsa = TruncatedSVD(n_components=num_components, n_iter=100, random_state=42)

# Fit SVD model on data
lsa.fit_transform(X)

# Get Singular values and Components 
Sigma = lsa.singular_values_ 
V_transpose = lsa.components_.T

In [216]:
lsa.explained_variance_ratio_

array([3.90828416e-29, 1.00000000e+00])

In [212]:
list(enumerate(lsa.components_))[0][1].shape

(1818,)

In [213]:
# Print the topics with their terms
terms = vectorizer.get_feature_names()

In [214]:
for doc, component in enumerate(lsa.components_):
    zipped = zip(terms, component)
    top_terms_key=sorted(zipped, key = lambda t: t[1], reverse=True)[:5]
    top_terms_list=list(dict(top_terms_key).keys())
    print("Document "+str(doc)+": ",top_terms_list)

Document 0:  ['last', 'forward', 'know', 'consumption', 'continue']
Document 1:  ['consumption', 'continue', 'inflation', 'really', 'brand']


## Linear Discriminant Analysis (LDA)
--> supervised learning !

## Latent Dirichlet Allocation (LDA)
#to do how data cleanned / words removeds

In [48]:
!pip install gensim

     |████████████████████████████████| 24.0 MB 16.4 MB/s eta 0:00:01
     |████████████████████████████████| 58 kB 12.8 MB/s eta 0:00:01


In [49]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.datasets import make_multilabel_classification
import gensim
import string
from gensim import corpora
from gensim.corpora.dictionary import Dictionary

In [50]:
final_doc = [document.split() for document in corpus]

In [59]:
dictionary = corpora.Dictionary(final_doc)
DT_matrix = [dictionary.doc2bow(doc) for doc in final_doc]
Lda_object = gensim.models.ldamodel.LdaModel

In [74]:
len(DT_matrix[0])

1117

- num_topics (int, optional) – The number of requested latent topics to be extracted from the training corpus.

In [89]:
lda_model_1 = Lda_object(DT_matrix, num_topics=2, id2word = dictionary)
print(lda_model_1.show_topics(num_words=10))

[(0, '0.003*"last" + 0.002*"forward" + 0.002*"side" + 0.002*"term" + 0.002*"around" + 0.002*"sort" + 0.002*"marketing" + 0.002*"maybe" + 0.002*"within" + 0.002*"existing"'), (1, '0.003*"know" + 0.003*"demand" + 0.003*"way" + 0.003*"forward" + 0.002*"looking" + 0.002*"marketing" + 0.002*"prior" + 0.002*"little" + 0.002*"continue" + 0.002*"last"')]
